In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data_df = pd.read_csv('raw_data\MasterCSV.csv')
data_df.head(10)

,Unnamed: 0,cuisine_SP,aisle_SP,ingredients_SP
0,Pear-ginger upside-down cake,[],"['Baking', 'Baking', 'Milk, Eggs, Other Dairy'...","['low sodium baking powder', 'baking soda', 'b..."
1,Easy Chicken Cordon Bleu,[],"['Meat', 'Meat', 'Cheese', 'Spices and Seasoni...","['boneless skinless chicken breast', 'ham', 'c..."
2,Chicken 65,[],"['Meat', 'Spices and Seasonings', 'Ethnic Food...","['chicken breast', 'chili powder', 'ginger gar..."
3,Herb Roasted Chicken,[],"['Spices and Seasonings', 'Baking', 'Milk, Egg...","['bay leaves', 'golden brown sugar', 'butter',..."
4,Meatball Sliders,[],"['Spices and Seasonings', 'Pasta and Rice', 'P...","['bay leaves', 'breadcrumbs', 'marjoram', 'egg..."
5,Saffron Chicken Tikka,"['Indian', 'Asian']","['Meat', 'Spices and Seasonings', 'Savory Snac...","['chicken breast', 'cinnamon stick', 'seeds', ..."
6,Mediterrean Tuna Salad,[],"['Produce', 'Oil, Vinegar, Salad Dressing', 'C...","['arugula', 'balsamic vinegar', 'capers', 'kal..."
7,Citrus Beet Salad,[],"['Produce', 'Oil, Vinegar, Salad Dressing', 'O...","['red beet', 'olive oil', 'balsamic vinegar', ..."
8,Peach & Brown Sugar Pancakes,[],"['Baking', 'Milk, Eggs, Other Dairy', 'Milk, E...","['golden brown sugar', 'egg', 'milk', 'pancake..."
9,Chicken En Papillote With Basil and Cherry Tom...,[],"['Meat', 'Spices and Seasonings', 'Oil, Vinega...","['boneless skinless chicken breast', 'salt and..."


In [3]:
data_df.rename(columns={'Unnamed: 0': 'recipe_name'}, inplace=True)

In [4]:
data_df.head(10)

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Pear-ginger upside-down cake,[],"['Baking', 'Baking', 'Milk, Eggs, Other Dairy'...","['low sodium baking powder', 'baking soda', 'b..."
1,Easy Chicken Cordon Bleu,[],"['Meat', 'Meat', 'Cheese', 'Spices and Seasoni...","['boneless skinless chicken breast', 'ham', 'c..."
2,Chicken 65,[],"['Meat', 'Spices and Seasonings', 'Ethnic Food...","['chicken breast', 'chili powder', 'ginger gar..."
3,Herb Roasted Chicken,[],"['Spices and Seasonings', 'Baking', 'Milk, Egg...","['bay leaves', 'golden brown sugar', 'butter',..."
4,Meatball Sliders,[],"['Spices and Seasonings', 'Pasta and Rice', 'P...","['bay leaves', 'breadcrumbs', 'marjoram', 'egg..."
5,Saffron Chicken Tikka,"['Indian', 'Asian']","['Meat', 'Spices and Seasonings', 'Savory Snac...","['chicken breast', 'cinnamon stick', 'seeds', ..."
6,Mediterrean Tuna Salad,[],"['Produce', 'Oil, Vinegar, Salad Dressing', 'C...","['arugula', 'balsamic vinegar', 'capers', 'kal..."
7,Citrus Beet Salad,[],"['Produce', 'Oil, Vinegar, Salad Dressing', 'O...","['red beet', 'olive oil', 'balsamic vinegar', ..."
8,Peach & Brown Sugar Pancakes,[],"['Baking', 'Milk, Eggs, Other Dairy', 'Milk, E...","['golden brown sugar', 'egg', 'milk', 'pancake..."
9,Chicken En Papillote With Basil and Cherry Tom...,[],"['Meat', 'Spices and Seasonings', 'Oil, Vinega...","['boneless skinless chicken breast', 'salt and..."


In [5]:
# Converting strings to lists then remove [] and ()
def ingredients_cleanup (data_df, column):
    row_list = []
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        row = row.replace("[", '')
        row = row.replace("'", '')
        row = row.replace("]", '')
        row = row.replace('"', '')
        row_list = row.split( ',')
        data_df.loc[indexes[counter],column] = row_list
        counter += 1
    
    return data_df

In [6]:
# Removing leading white spaces 
def remove_leading_ws(data_df, column):
    indexes = data_df.index.values.tolist()
    counter = 0
    for row in data_df[column]:
        for i, element in enumerate(row):
            element = element.strip()
            data_df.loc[indexes[counter],column][i] = element
        counter += 1
    return data_df

In [7]:
# Creating a single list from all the ingredents 
def total_ing_list_from_df(data_df):
    composite_ingredients = []
    for row in data_df.ingredients_SP:
        for element in row:
            composite_ingredients.append(element)
    return composite_ingredients

In [8]:
# Creating list of composite list and creat count of ingredients
def sort_and_count_ingredient_list(ingredient_list):
    ingredient_df = pd.DataFrame(ingredient_list)
    ingredient_df = ingredient_df.rename(columns = {0:  'ingredient'})
    ingredient_df['count'] = 1
    ingredient_counts = ingredient_df.groupby('ingredient').agg({'count':  'count'})
    ingredient_counts.sort_values('count', ascending=False, inplace=True)
    return ingredient_counts


In [9]:
# Replacing different verisons of ingredients
def ingredient_replacement (data_df, replacement_dict):
    
    
    axises = data_df.index.values.tolist()
    counter = 0
    for row in data_df.ingredients_SP:
        for i, element in enumerate(row):
            if element in replacement_dict.keys():
                element = replacement_dict[element]
                data_df.loc[axises[counter],'ingredients_SP'][i] = element
        counter += 1
    return data_df

In [10]:
# Calling ingredient clean up function to create list
data_df = ingredients_cleanup(data_df, 'ingredients_SP')
data_df = ingredients_cleanup(data_df, 'aisle_SP')
data_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Pear-ginger upside-down cake,[],"[Baking, Baking, Milk, Eggs, Other Dairy, ...","[low sodium baking powder, baking soda, butt..."
1,Easy Chicken Cordon Bleu,[],"[Meat, Meat, Cheese, Spices and Seasonings,...","[boneless skinless chicken breast, ham, ched..."
2,Chicken 65,[],"[Meat, Spices and Seasonings, Ethnic Foods, ...","[chicken breast, chili powder, ginger garlic..."
3,Herb Roasted Chicken,[],"[Spices and Seasonings, Baking, Milk, Eggs,...","[bay leaves, golden brown sugar, butter, dr..."
4,Meatball Sliders,[],"[Spices and Seasonings, Pasta and Rice, Prod...","[bay leaves, breadcrumbs, marjoram, egg, p..."
...,...,...,...,...
9995,Cinnamon Twists,[],"[Baking, Milk, Eggs, Other Dairy, Milk, E...","[dry yeast, butter, egg, milk, salt, sugar]"
9996,Fluffy frittata with spinach,[],"[Milk, Eggs, Other Dairy, Produce, Spices ...","[egg, garlic, black pepper, nutmeg, olive ..."
9997,Protein Packed Carrot Muffins,[],"[Spices and Seasonings, Gluten Free;Health Fo...","[dry seasoning rub, almond meal, low sodium ..."
9998,BLT Sandwich,[],"[Produce, Bakery/Bread, Produce, Condiments...","[bell pepper, bread, lettuce, mayonnaise, ..."


In [11]:
data_df = remove_leading_ws(data_df, 'ingredients_SP')

In [12]:
data_df.head()

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Pear-ginger upside-down cake,[],"[Baking, Baking, Milk, Eggs, Other Dairy, ...","[low sodium baking powder, baking soda, butter..."
1,Easy Chicken Cordon Bleu,[],"[Meat, Meat, Cheese, Spices and Seasonings,...","[boneless skinless chicken breast, ham, chedda..."
2,Chicken 65,[],"[Meat, Spices and Seasonings, Ethnic Foods, ...","[chicken breast, chili powder, ginger garlic p..."
3,Herb Roasted Chicken,[],"[Spices and Seasonings, Baking, Milk, Eggs,...","[bay leaves, golden brown sugar, butter, dried..."
4,Meatball Sliders,[],"[Spices and Seasonings, Pasta and Rice, Prod...","[bay leaves, breadcrumbs, marjoram, egg, parsl..."


In [14]:
composite_ingredients = total_ing_list_from_df(data_df)

In [15]:
ing_groups = sort_and_count_ingredient_list(composite_ingredients)
ing_groups.to_csv('ingredient_list_master_0909.csv')

In [16]:
replacement_dict = {'coarse sea salt':  'salt_and_pepper',
                    'salsa verde':  'salsa',
                    'boneless skinless chicken breast':  'chicken',
                    'salt':  'salt and pepper',
                    'whole chicken':  'chicken',
                    'white onion':  'onion',
                    'solive oil':  'oil',
                    'palm oil':   'oil',
                    'dried arbol chile':  'chili pepper',
                    'cooking oil':  'oil',
                    'cooked chicken':  'chicken',
                    'yellow onion':  'onion',
                    'ripe banana':'banana',
                    'fresh basil':'basil',
                    'black beans':'beans',
                    'canned kidney beans':'beans',
                    'canned pinto beans':'beans',
                    'chili beans':'beans',
                    'dried black beans':'beans',
                    'refried beans':'beans',
                    '95 percent lean ground beef':'beef',
                    'beef cubes':'beef',
                    'beef shoulder roast':'beef',
                    'flank steak':'beef',
                    'ground chuck':'beef',
                    'round tip steak':'beef',
                    'skirt steak':'beef',
                    'steak':'beef',
                    'green pepper':'bell pepper',
                    'orange pepper':'bell pepper',
                    'red pepper':'bell pepper',
                    'roasted red peppers':'bell pepper',
                    'yellow pepper':'bell pepper',
                    'canned black beans':'beans',
                    'french bread':'bread',
                    'beef broth':'broth',
                    'chicken broth':'broth',
                    'chicken stock':'broth',
                    'low sodium chicken broth':'broth',
                    'red cabbage':'cabbage',
                    'canned chipotle chile':'canned pepper',
                    'canned green chiles':'canned pepper',
                    'chipotle chiles':'canned pepper',
                    'peppers':'fresh peppers',
                    'ground cayenne pepper':'cayenne',
                    'cheddar cheese':'cheese',
                    'colby jack':'cheese',
                    'cotija cheese':'cheese',
                    'cream cheese':'cheese',
                    'extra sharp cheddar cheese':'cheese',
                    'feta cheese':'cheese',
                    'goat cheese':'cheese',
                    'grilling cheese':'cheese',
                    'low fat shredded cheddar':'cheese',
                    'mexican cheese':'cheese',
                    'monterey jack cheese':'cheese',
                    'queso fresco':'cheese',
                    'sharp cheddar cheese':'cheese',
                    'shredded cheddar cheese':'cheese',
                    'shredded cheese':'cheese',
                    'shredded mexican cheese blend':'cheese',
                    'soy cheese':'cheese',
                    'boneless skinless chicken breast':'chicken',
                    'boneless skinless chicken thighs':'chicken',
                    'chicken breast':'chicken',
                    'chicken breast halves':'chicken',
                    'chicken meat':'chicken',
                    'rotisserie chicken':'chicken',
                    'shredded chicken':'chicken',
                    'whole chicken':'chicken',
                    'condensed cream of chicken soup':'chicken soup',
                    'ancho chili powder':'chili powder',
                    'chili pepper':'chili powder',
                    'chili powder':'chili powder',
                    'chili seasoning mix':'chili powder',
                    'ground chipotle chile pepper':'chipotle',
                    'dried cilantro':'cilantro',
                    'cod fillets':'fish',
                    'ground coriander':'coriander',
                    'corn kernels':'corn',
                    'creamed corn':'corn',
                    'frozen corn':'corn',
                    'sweet corn on the cob':'corn',
                    'whole kernel corn':'corn',
                    'mexican crema':'crema',
                    'creole seasoning':'creole season',
                    'persian cucumber':'cucumber',
                    'cumin seeds':'cumin',
                    'ground cumin':'cumin',
                    'dried arbol chile':'dried pepper',
                    'green enchilada sauce':'enchilada sauce',
                    'salmon':'fish',
                    'swordfish':'fish',
                    'tilapia':'fish',
                    'anaheim pepper':'fresh pepper',
                    'green chili pepper':'fresh pepper',
                    'jalapeno pepper':'fresh pepper',
                    'pasilla pepper':'fresh pepper',
                    'poblano pepper':'fresh pepper',
                    'serrano pepper':'fresh pepper',
                    'garlic powder':'garlic',
                    'garlic salt':'garlic',
                    'whole garlic cloves':'garlic',
                    'ginger':'ginger',
                    'guacamole':'guacamole',
                    'smoked ham hock':'ham hock',
                    'light vanilla ice cream':'ice cream',
                    'italian seasoning':'italian season',
                    'lemon juice':'lemon',
                    'lemon peel':'lemon',
                    'butter lettuce leaves':'lettuce',
                    'romaine':'lettuce',
                    'lime juice':'lime',
                    'lime peel':'lime',
                    'lime wedge':'lime',
                    'dried mango':'mango',
                    'condensed cream of mushroom soup':'mushroom soup',
                    'fresh mushrooms':'mushrooms',
                    'mustard powder':'mustard powder',
                    'coconut oil':'oil',
                    'cooking oil':'oil',
                    'extra virgin olive oil':'oil',
                    'grape seed oil':'oil',
                    'light olive oil':'oil',
                    'olive oil':'oil',
                    'peanut oil':'oil',
                    'rice bran oil':'oil',
                    'vegetable oil':'oil',
                    'black olives':'olives',
                    'olives':'olives',
                    'onion powder':'onion',
                    'red onion':'onion',
                    'white onion':'onion',
                    'yellow onion':'onion',
                    'spring onions':'onion',
                    'navel orange':'orange',
                    'orange juice':'orange',
                    'smoked paprika':'paprika',
                    'peas and carrots':'pea',
                    'pico de gallo':'pico de gallo',
                    'pistachio nuts':'pistachio',
                    'boston butt':'pork',
                    'lean pork tenderloin':'pork',
                    'pork chops':'pork',
                    'ground pork sausage':'pork sausage',
                    'yukon gold potato':'potato',
                    'cooked quinoa':'quinoa',
                    'red pepper flakes':'red pepper flake',
                    'brown rice':'rice',
                    'cooked brown rice':'rice',
                    'cooked rice':'rice',
                    'picante sauce':'salsa',
                    'salsa verde':'salsa',
                    'kosher salt':'salt and pepper',
                    'black pepper':'salt and pepper',
                    'coarse sea salt':'salt and pepper',
                    'salt':'salt and pepper',
                    'seasoned salt':'salt and pepper',
                    'white pepper':'salt and pepper',
                    'vegetable stock':'broth',
                    'dark brown sugar':'sugar',
                    'golden brown sugar':'sugar',
                    'dried thyme':'thyme',
                    'canned diced tomatoes':'tomato',
                    'canned tomatoes':'tomato',
                    'cherry tomato':'tomato',
                    'diced tomatoes with green chilies':'tomato',
                    'fire roasted tomatoes':'tomato',
                    'italian tomato':'tomato',
                    'plum tomato':'tomato',
                    'tomato juice':'tomato',
                    'tomato paste':'tomato',
                    'canned tomato sauce':'tomato sauce',
                    'burrito size tortilla':'tortilla',
                    'flour tortilla':'tortilla',
                    'scoops tortilla chips':'tortilla',
                    'white corn tortilla':'tortilla',
                    'wrap':'tortilla',
                    'corn tortilla chips':'tortilla',
                    'ground turkey':'turkey',
                    'apple cider vinegar':'vinegar',
                    'balsamic vinegar':'vinegar',
                    'distilled white vinegar':'vinegar',
                    'white wine vinegar':'vinegar',
                    'fat free greek yogurt':'yogurt',
                    'greek yogurt':'yogurt',
                    'low fat yogurt':'yogurt',

}

In [17]:
data_df = ingredient_replacement(data_df, replacement_dict)
data_df

,recipe_name,cuisine_SP,aisle_SP,ingredients_SP
0,Pear-ginger upside-down cake,[],"[Baking, Baking, Milk, Eggs, Other Dairy, ...","[low sodium baking powder, baking soda, butter..."
1,Easy Chicken Cordon Bleu,[],"[Meat, Meat, Cheese, Spices and Seasonings,...","[chicken, ham, cheese, salt and pepper, black ..."
2,Chicken 65,[],"[Meat, Spices and Seasonings, Ethnic Foods, ...","[chicken, chili powder, ginger garlic paste, s..."
3,Herb Roasted Chicken,[],"[Spices and Seasonings, Baking, Milk, Eggs,...","[bay leaves, sugar, butter, dried basil, rosem..."
4,Meatball Sliders,[],"[Spices and Seasonings, Pasta and Rice, Prod...","[bay leaves, breadcrumbs, marjoram, egg, parsl..."
...,...,...,...,...
9995,Cinnamon Twists,[],"[Baking, Milk, Eggs, Other Dairy, Milk, E...","[dry yeast, butter, egg, milk, salt and pepper..."
9996,Fluffy frittata with spinach,[],"[Milk, Eggs, Other Dairy, Produce, Spices ...","[egg, garlic, salt and pepper, nutmeg, oil, pa..."
9997,Protein Packed Carrot Muffins,[],"[Spices and Seasonings, Gluten Free;Health Fo...","[dry seasoning rub, almond meal, low sodium ba..."
9998,BLT Sandwich,[],"[Produce, Bakery/Bread, Produce, Condiments...","[bell pepper, bread, lettuce, mayonnaise, thic..."


In [18]:
composite_ingredients = total_ing_list_from_df(data_df)

In [19]:
ing_groups = sort_and_count_ingredient_list(composite_ingredients)
ing_groups.head(20)

,count
ingredient,
salt and pepper,7901
sugar,4343
oil,4197
onion,3189
garlic,3103
egg,2951
butter,2513
wheat flour,2321
lemon,1972


In [20]:
# Dropping common ingredients 
ing_groups = ing_groups.drop(index = ["salt and pepper", "oil"])
ing_groups

,count
ingredient,
sugar,4343
onion,3189
garlic,3103
egg,2951
butter,2513
...,...
low fat vanilla yogurt,4
pimenton de la vera,4
mulberries,3
